In [ ]:
import os
import sys

# Get the current working directory and add the parent directory to the Python path
current_working_directory = os.getcwd()
print(os.path.join(current_working_directory, ".."))
sys.path.append(os.path.join(current_working_directory, ".."))

In [ ]:
from decouple import AutoConfig
config = AutoConfig(search_path='./../.env')

In [ ]:
os.environ["AZURE_OPENAI_API_KEY"] = config('OPENAI_API_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = config('AZURE_ENDPOINT')

## Functions, Models, and Chains

### Function Binding

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from models.llm import LLM

In [ ]:
def joke_generator(topic):
    "This function will return a joke based on the topic provided"
    model = LLM('gpt-4')
    llm = model.load_model()
    prompt = ChatPromptTemplate.from_template(
        "Write a short joke on {topic}"
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke({"topic":topic})
    

In [ ]:
response = joke_generator("Artificial Intelligence")
print(response)

In [ ]:
def poem_generator(topic):
    "This function will return a poem based on the topic provided"
    model = LLM('gpt-4')
    llm = model.load_model()
    prompt = ChatPromptTemplate.from_template(
        "Write a short poem on {topic}"
    )
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    return chain.invoke({"topic":topic})

In [ ]:
response = poem_generator("Artificial Intelligence")
print(response)

#### OpenAI functon format

In [ ]:
functions =[
    {
      "name": "joke_generator",
      "description": "Generates a joke based on the topic provided",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The topic to get the joke for"
          },
        },
        "required": ["topic"]
      }
    },
    {
      "name": "poem_generator",
      "description": "Generates a poem based on the topic provided",
      "parameters": {
        "type": "object",
        "properties": {
          "topic": {
            "type": "string",
            "description": "The topic to get the poem for"
          },
        },
        "required": ["topic"]
      }
    }
]

#### Attaching functions with model invocation

In [ ]:
messages = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a smart and intelligent AI Assistant."),
        ("user", "{input}")
    ]
)

In [ ]:
from langchain_openai import AzureChatOpenAI
gpt4 = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            )

In [ ]:
gpt4.invoke(input="poem on Artificial Intelligence", functions=functions)

**NOTE:** With this approach, it is required to attach functions with everytime model is invoked.

#### Binding model with functions

In [ ]:
from langchain_openai import AzureChatOpenAI
gpt4 = AzureChatOpenAI(
                openai_api_version=config('AZURE_CHAT_OPENAI_API_VERSION'),
                azure_deployment=config('AZURE_GPT4_CHAT_OPENAI_DEPLOYMENT'),
                temperature=0,
                max_tokens=1024
            ).bind(functions=functions)

In [ ]:
runnable = messages | gpt4

In [ ]:
runnable.invoke({"input": "joke about Artificial Intelligence"})

In [ ]:
runnable.invoke({"input": "poem on Artificial Intelligence"})